# VWAP Mean Reversion dengan ATR Guardrail

Backtest singkat memakai fixture `mini_ohlcv`.

In [1]:
from __future__ import annotations

import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import pandas as pd

from src.strategy_backtest.pipeline import SignalBacktester
from src.strategy_backtest.strategies.vwap import Strategy as VWAPStrategy


In [2]:
DATA_PATH = PROJECT_ROOT / 'tests/fixtures/mini_ohlcv.csv'
df = pd.read_csv(DATA_PATH)
df['time'] = pd.to_datetime(df['time'], unit='s')
df = df.set_index('time')
df.columns = [c.lower() for c in df.columns]
df = df[['open', 'high', 'low', 'close', 'volume']].astype(float)
df.head()


,open,high,low,close,volume
time,,,,,
2022-01-01,1434.243188,1444.943285,1429.421824,1432.450712,23339.966868
2022-01-02,1357.876212,1368.143066,1353.429203,1360.208037,25926.747744
2022-01-03,1300.799170,1316.133978,1299.241305,1303.591182,31495.824118
2022-01-04,1263.197589,1267.932945,1250.741573,1261.979848,46794.224657
2022-01-05,1200.183368,1204.150603,1193.059170,1200.720001,23853.839775


In [3]:
strategy = VWAPStrategy(rsi_length=5, atr_length=5, atr_stop_multiplier=1.2, atr_take_profit_multiplier=2.0, session_frequency='1D')
signals = strategy.generate_signals(df)
signals[['long_entry','long_exit','short_entry','short_exit','stop_level','take_profit_level']].head()


,long_entry,long_exit,short_entry,short_exit,stop_level,take_profit_level
time,,,,,,
2022-01-01,False,False,False,False,NaN,NaN
2022-01-02,False,False,False,False,NaN,NaN
2022-01-03,False,False,False,False,NaN,NaN
2022-01-04,False,False,False,False,NaN,NaN
2022-01-05,False,False,False,False,NaN,NaN


In [4]:
backtester = SignalBacktester(df, price_column='close')
outputs = backtester.run(signals)
pd.Series(outputs.metrics)


total_return             0.028492
cagr                     0.058622
sharpe_ratio             2.052183
annualised_vol           0.027951
max_drawdown            -0.005208
avg_drawdown_duration    2.000000
dtype: float64